# Ensemble Learning - XGBoost 

### 강의 자료 속 나온 3개의 수도 코드 구현에 집중 

### Algorithm 1 : Exact Greedy Algorithm for Split Finding 

Input : I, instance set of current node 
Input : d, feature dimension 

1. gain <- 0 
2. G $\leftarrow \sum_{i \in I}g_i$, H $\leftarrow \sum_{i \in I} h_i$ 
3. for k = 1 to m do :
- $G_L \leftarrow 0, H_L \leftarrow 0$ 
- for j in sorted(I, by $x_{jk}$) do 

> $G_L \leftarrow G_L + g_j, H_L \leftarrow H_L + h_j$

> $G_R \leftarrow G - G_L, H_R \leftarrow H - H_L$ 

> score $\leftarrow$ max(score, $\frac{G_L^2}{H_L + \lambda} + \frac{G_R^2}{H_R + \lambda} - \frac{G^2}{H + \lambda}) $ 

> end 
- end 

output : Split with max score 

**구현해야 하는 것**
- Loss function : $l(y_i, \hat y_i^{t-1} + f_t(x_i))$ 
- $G_L, G_R$ : 손실함수의 1차 미분의 좌측 합, 1차 미분의 나머지 합 
- $H_L, H_R$ : 손실함수의 2차 미분의 좌측 합, 2차 미분의 나머지 합 
- $\lambda$ : $\rho(f_k) = \gamma T + \frac{1}{2} \lambda ||w||^2$ 에서의 $\lambda$ 값
> $\gamma, \lambda$ : Parameter.

**필요한 것** 
- CART Tree : DT Regression 적용하겠음. 
- Loss 함수 정의 : 여기선 간단한 OLS을 채택하겠음. 

> g = f(x) - y : 예측값 - 실제값 

> h = 1 : 상수 

> f(x) = DT Regression으로 구하기.  

**함수의 형태** 
- def __init__(self, X, y, gamma, lambda) : 

- def altorithm_1(self) : > 모든 Split point 계산하기




In [98]:
import numpy as np
import random as rand
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()

X = load_iris()['data']
y = load_iris()["target"]

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state=10)

In [96]:
# 다른 데이터 셋 체크 

from sklearn.datasets import load_diabetes 
from sklearn.model_selection import train_test_split 

X,y = load_diabetes(return_X_y = True)
train_x, test_x, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state=10)

In [100]:
class XGBoost() : 
    def __init__(self, X, y, num_repeat,  gamma, lamda) : 
        self.X = X 
        self.n = np.shape(X)[0]
        self.d = np.shape(X)[1]
        
        self.y = y 
        self.num_label = len(np.unique(y))
        
        self.num_repeat = num_repeat
        
        self.gamma = gamma 
        self.lamda = lamda
    
    
    def algorithm_1(self, X, y) : 
        # gain 변수는 사용하지 않아 Score 변수로 변경하여 사용 
        score = 0 
        target_value = y
    
        g_lst = [] 
        h_lst = [] 
        
        for i in range(self.num_repeat) : 
            model = tree.DecisionTreeRegressor()
            model.fit(self.X, self.y)
            pred_y = model.predict(X)
            
            grad = np.array(pred_y) - np.array(target_value)
            h = np.ones(len(X))
            g_lst.append(grad)
            h_lst.append(h)
            target_value = grad 
        
        G = np.sum(g_lst) 
        H = np.sum(h_lst)
        
        
        split_point = 0 
        att = 0
        
        for k in range(self.d) : 
            G_left = 0 
            H_left = 0 

            # j 는 각 특성별 크기에 sorting 된 순서
            for j in np.argsort(np.array(X)[:,k]) : 
            # 각 객체별로 g, h의 값을 더하기. 
                G_left += np.sum([g_lst[k][j] for k in range(len(g_lst))]) 
                H_left += np.sum([h_lst[k][j] for k in range(len(h_lst))])
                G_right = G - G_left
                H_right = H - H_left 
            
                cur_score = (G_left**2 /(H_left + self.lamda)) + (G_right**2/(H_right + self.lamda)) - (G**2/(H+self.lamda))
                if score <= cur_score : 
                    score = cur_score
                    split_point= j 
                    att = k
            
        index = np.argsort(np.array(X)[:, att])
        X,y = X[index], y[index] 
        return X[:split_point+1], y[:split_point+1], X[split_point+1:], y[split_point+1:] 

In [103]:
test = XGBoost(train_x,train_y, 5, 1, 2)
a_input, a_target, b_input, b_target = test.algorithm_1(test_x,test_y)
test.algorithm_1(b_input, b_target)

(array([[5.9, 3. , 4.2, 1.5],
        [5.6, 3. , 4.5, 1.5],
        [6. , 3.4, 4.5, 1.6],
        [6.2, 2.8, 4.8, 1.8]]),
 array([1, 1, 1, 2]),
 array([[6.3, 2.5, 4.9, 1.5],
        [5.8, 2.7, 5.1, 1.9],
        [6.4, 2.7, 5.3, 1.9],
        [6.8, 3. , 5.5, 2.1],
        [6.4, 2.8, 5.6, 2.2],
        [7.1, 3. , 5.9, 2.1],
        [7.7, 3.8, 6.7, 2.2]]),
 array([1, 2, 2, 2, 2, 2, 2]))

### Algorithm 2 : Approximate Algorithm for split finding 

1. for k = 1 to m do 
- Propose $S_k$ = ${s_{k1}, s_{k2}, ... s_{kl}}$ by percentiles on feature k. 
- proposal can be done per tree (global), or per split(local) 
- end 

2. for k =1 to m do 
- $G_{kv}$ = $\sum_{j \in {j|s_k, v >= x_{jk} > s_k, v-1}} g_j$ 

- $H_{kv} = \sum_{j \in {j|s_k, v >= x_{jk} > s_k, v-1}} h_j$ 
- end 

3. Follow same step as in previous section to find max score only among proposed splits. 

In [215]:
class XGBoost() : 
    def __init__(self, X, y, num_repeat,  gamma, lamda) : 
        self.X = X 
        self.n = np.shape(X)[0]
        self.d = np.shape(X)[1]
        
        self.y = y 
        self.num_label = len(np.unique(y))
        
        self.num_repeat = num_repeat
        
        self.gamma = gamma 
        self.lamda = lamda

    def algorithm_1(self, X, y) : 
        # gain 변수는 사용하지 않아 Score 변수로 변경하여 사용 
        score = 0 
        target_value = y
    
        g_lst = [] 
        h_lst = [] 
        
        for i in range(self.num_repeat) : 
            model = tree.DecisionTreeRegressor()
            model.fit(self.X, self.y)
            pred_y = model.predict(X)
            
            grad = np.array(pred_y) - np.array(target_value)
            h = np.ones(len(X))
            g_lst.append(grad)
            h_lst.append(h)
            target_value = grad 
        
        G = np.sum(g_lst) 
        H = np.sum(h_lst)
        
        
        split_point = 0 
        att = 0
        
        for k in range(self.d) : 
            G_left = 0 
            H_left = 0 

            # j 는 각 특성별 크기에 sorting 된 순서
            for j in np.argsort(np.array(X)[:,k]) : 
            # 각 객체별로 g, h의 값을 더하기. 
                G_left += np.sum([g_lst[k][j] for k in range(len(g_lst))]) 
                H_left += np.sum([h_lst[k][j] for k in range(len(h_lst))])
                G_right = G - G_left
                H_right = H - H_left 
            
                cur_score = (G_left**2 /(H_left + self.lamda)) + (G_right**2/(H_right + self.lamda)) - (G**2/(H+self.lamda))
                if score <= cur_score : 
                    score = cur_score
                    split_point= j 
                    att = k
            
        index = np.argsort(np.array(X)[:, att])
        X,y = X[index], y[index] 
        return X[:split_point+1], y[:split_point+1], X[split_point+1:], y[split_point+1:] 
        
 

    def algorithm_2(self,X,y, typ, bucket_size) :
        score = 0 
        target_value = y
    
        g_lst = [] 
        h_lst = [] 
        
        for i in range(self.num_repeat) : 
            model = tree.DecisionTreeRegressor()
            model.fit(self.X, self.y)
            pred_y = model.predict(X)
            
            grad = np.array(pred_y) - np.array(target_value)
            h = np.ones(len(X))
            g_lst.append(grad)
            h_lst.append(h)
            target_value = grad 
        
        G = np.sum(g_lst) 
        H = np.sum(h_lst)
        
        # Sk list를 표현할 index 뽑기 
        index_lst = [] 
        split_size = len(X) / bucket_size
        for k in range(self.d) :
            index = np.argsort(np.array(X)[:,k])
            index_att_lst = [] 
            for per in range(bucket_size) : 
                per_index = list(np.where((split_size*(per+1) > index) &(index >= split_size*(per)))[0])
                index_att_lst.append(per_index)
            index_lst.append(index_att_lst)
        
        #G_kv, H_kv 구하기 
        G_kv = [] 
        H_kv = [] 
        bucket_lst = [] 
        for bucket in range(bucket_size) : 
            bucket_index = [index_lst[k][bucket] for k in range(self.d)]
            bucket_lst.append(bucket_index)
        
        g_sum_lst = np.array(np.sum(g_lst,axis =0))
        h_sum_lst = np.array(np.sum(h_lst, axis =0 ))
        for k in range(self.d) :
            bucket_index = [bucket_lst[bucket][k] for bucket in range(bucket_size)]
            G_kv.append([np.sum(g_sum_lst[np.array(bucket_index[bucket])]) for bucket in range(bucket_size)])
            H_kv.append([np.sum(h_sum_lst[np.array(bucket_index[bucket])]) for bucket in range(bucket_size)])
                        
    # 뒷 과정은 생략. Sk 로 구분한 데이터 셋을 algorithm1에 적용해도 됨. 
            



### Algorithm 3 : Sparsity-Aware Split finding 

Missing Value를 좌, 우 측에 붙여야 한다. 

이를 위해서 Missing value 데이터와 그렇지 않은 데이터를 구분한다. 
그리고 모든 특성 값이 있는 데이터를 오름차순, 내림차순으로 정렬한 다음, Missing value 데이터를 각각 좌, 우측으로 붙인다.


In [ ]:
class XGBoost() : 
    def __init__(self, X, y, num_repeat,  gamma, lamda) : 
        self.X = X 
        self.n = np.shape(X)[0]
        self.d = np.shape(X)[1]
        
        self.y = y 
        self.num_label = len(np.unique(y))
        
        self.num_repeat = num_repeat
        
        self.gamma = gamma 
        self.lamda = lamda

    def algorithm_1(self, X, y) : 
        # gain 변수는 사용하지 않아 Score 변수로 변경하여 사용 
        score = 0 
        target_value = y
    
        g_lst = [] 
        h_lst = [] 
        
        for i in range(self.num_repeat) : 
            model = tree.DecisionTreeRegressor()
            model.fit(self.X, self.y)
            pred_y = model.predict(X)
            
            grad = np.array(pred_y) - np.array(target_value)
            h = np.ones(len(X))
            g_lst.append(grad)
            h_lst.append(h)
            target_value = grad 
        
        G = np.sum(g_lst) 
        H = np.sum(h_lst)
        
        
        split_point = 0 
        att = 0
        
        for k in range(self.d) : 
            G_left = 0 
            H_left = 0 

            # j 는 각 특성별 크기에 sorting 된 순서
            for j in np.argsort(np.array(X)[:,k]) : 
            # 각 객체별로 g, h의 값을 더하기. 
                G_left += np.sum([g_lst[k][j] for k in range(len(g_lst))]) 
                H_left += np.sum([h_lst[k][j] for k in range(len(h_lst))])
                G_right = G - G_left
                H_right = H - H_left 
            
                cur_score = (G_left**2 /(H_left + self.lamda)) + (G_right**2/(H_right + self.lamda)) - (G**2/(H+self.lamda))
                if score <= cur_score : 
                    score = cur_score
                    split_point= j 
                    att = k
            
        index = np.argsort(np.array(X)[:, att])
        X,y = X[index], y[index] 
        return X[:split_point+1], y[:split_point+1], X[split_point+1:], y[split_point+1:] 
        
 

    def algorithm_2(self,X,y, typ, bucket_size) :
        score = 0 
        target_value = y
    
        g_lst = [] 
        h_lst = [] 
        
        for i in range(self.num_repeat) : 
            model = tree.DecisionTreeRegressor()
            model.fit(self.X, self.y)
            pred_y = model.predict(X)
            
            grad = np.array(pred_y) - np.array(target_value)
            h = np.ones(len(X))
            g_lst.append(grad)
            h_lst.append(h)
            target_value = grad 
        
        G = np.sum(g_lst) 
        H = np.sum(h_lst)
        
        # Sk list를 표현할 index 뽑기 
        index_lst = [] 
        split_size = len(X) / bucket_size
        for k in range(self.d) :
            index = np.argsort(np.array(X)[:,k])
            index_att_lst = [] 
            for per in range(bucket_size) : 
                per_index = list(np.where((split_size*(per+1) > index) &(index >= split_size*(per)))[0])
                index_att_lst.append(per_index)
            index_lst.append(index_att_lst)
        
        #G_kv, H_kv 구하기 
        G_kv = [] 
        H_kv = [] 
        bucket_lst = [] 
        for bucket in range(bucket_size) : 
            bucket_index = [index_lst[k][bucket] for k in range(self.d)]
            bucket_lst.append(bucket_index)
        
        g_sum_lst = np.array(np.sum(g_lst,axis =0))
        h_sum_lst = np.array(np.sum(h_lst, axis =0 ))
        for k in range(self.d) :
            bucket_index = [bucket_lst[bucket][k] for bucket in range(bucket_size)]
            G_kv.append([np.sum(g_sum_lst[np.array(bucket_index[bucket])]) for bucket in range(bucket_size)])
            H_kv.append([np.sum(h_sum_lst[np.array(bucket_index[bucket])]) for bucket in range(bucket_size)])
                        
    # 뒷 과정은 생략. Sk 로 구분한 데이터 셋을 algorithm1에 적용해도 됨. 
    
    def algorithm_3(self, X, y, missing_X, missing_y) : 
        # gain 변수는 사용하지 않아 Score 변수로 변경하여 사용 
        score = 0 
        target_value = y + missing_y
    
        g_lst = [] 
        h_lst = [] 
        
        for i in range(self.num_repeat) : 
            model = tree.DecisionTreeRegressor()
            model.fit(self.X, self.y)
            
            S_X = X + missing_X 
            S_y = y + missing_y
            pred_y = model.predict(S_X)
            
            grad = np.array(pred_y) - np.array(target_value)
            h = np.ones(len(S_X))
            g_lst.append(grad)
            h_lst.append(h)
            target_value = grad 
        
        G = np.sum(g_lst) 
        H = np.sum(h_lst)
        
        
        split_point = 0 
        att = 0

        #  Missing value 오른쪽으로 몰기 
        for k in range(self.d) : 
            G_left = 0 
            H_left = 0 

            for j in np.argsort(np.array(X)[:,k]) : 
            # 각 객체별로 g, h의 값을 더하기. 
                G_left += np.sum([g_lst[k][j] for k in range(len(g_lst))]) 
                H_left += np.sum([h_lst[k][j] for k in range(len(h_lst))])
                G_right = G - G_left
                H_right = H - H_left 
            
                cur_score = (G_left**2 /(H_left + self.lamda)) + (G_right**2/(H_right + self.lamda)) - (G**2/(H+self.lamda))
                if score <= cur_score : 
                    score = cur_score
                    split_point= j 
                    att = k

        for k in range(self.d) : 
            G_right = 0 
            H_right = 0 
            
            for j in np.argsort(np.array(X)[:,k]) : 
            # 각 객체별로 g, h의 값을 더하기. 
                G_right += np.sum([g_lst[k][-j] for k in range(len(g_lst))]) 
                H_right += np.sum([h_lst[k][-j] for k in range(len(h_lst))])
                G_left = G - G_right
                H_left = H - H_right
            
                cur_score = (G_left**2 /(H_left + self.lamda)) + (G_right**2/(H_right + self.lamda)) - (G**2/(H+self.lamda))
                if score <= cur_score : 
                    score = cur_score
                    split_point= j 
                    att = k
                    
            
            
        index = np.argsort(np.array(X)[:, att])
        X,y = X[index], y[index] 
        return X[:split_point+1], y[:split_point+1], X[split_point+1:], y[split_point+1:] 


In [219]:
a = [[1,2,3]]
b = [[2,3,4]]

a + b


[[1, 2, 3], [2, 3, 4]]